In [1]:
%load_ext sql
import pandas as pd
import sqlite3

In [2]:
%sql sqlite:///CourseData.db
conn = sqlite3.connect('CourseData.db')

# ERD

# Create Tables to Represent ERD

In [3]:
%%sql

-- Programs table
DROP TABLE IF EXISTS PROGRAMS;
CREATE TABLE PROGRAMS (
    ProgramID INTEGER PRIMARY KEY,
    ProgramCode TEXT NOT NULL,
    ProgramName TEXT NOT NULL
);

-- Course Catalogs table
DROP TABLE IF EXISTS CATALOG_COURSES;
CREATE TABLE CATALOG_COURSES (
    CourseID INTEGER PRIMARY KEY,
    CatalogYear TEXT NOT NULL,
    CatalogID TEXT NOT NULL,
    ProgramID INTEGER,
    CourseTitle TEXT NOT NULL,
    Credits TEXT NOT NULL,
    Prereqs TEXT,
    Coreqs TEXT,
    Fees TEXT,
    Attributes TEXT,
    Description TEXT,
    FOREIGN KEY (ProgramID) REFERENCES PROGRAMS(ProgramID)
);
CREATE INDEX ix_catalog_courses_alt_key on CATALOG_COURSES(CatalogYear,CatalogID);

-- Instructors table
DROP TABLE IF EXISTS INSTRUCTORS;
CREATE TABLE INSTRUCTORS (
    InstructorID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);
CREATE INDEX ix_instructors_name on INSTRUCTORS(Name);

-- Course Offerings table
DROP TABLE IF EXISTS COURSE_OFFERINGS;
CREATE TABLE COURSE_OFFERINGS (
    CourseOfferingID INTEGER PRIMARY KEY,
    CourseID INTEGER,
    CatalogID TEXT NOT NULL,
    Term TEXT NOT NULL,
    CRN INTEGER NOT NULL,
    Section TEXT NOT NULL,
    Credits REAL,
    Title TEXT NOT NULL,
    Timecodes TEXT,
    PrimaryInstructorID INTEGER,
    Capacity INTEGER NOT NULL,
    Actual INTEGER NOT NULL,
    Remaining INTEGER NOT NULL,
    FOREIGN KEY (CourseID) REFERENCES CATALOG_COURSES(CourseID),
    FOREIGN KEY (PrimaryInstructorID) REFERENCES INSTRUCTORS(InstructorID)
);
CREATE INDEX ix_course_offerings_alt_key on COURSE_OFFERINGS(Term,CatalogID,Section);

-- Locations table
DROP TABLE IF EXISTS LOCATIONS;
CREATE TABLE LOCATIONS (
    LocationID INTEGER PRIMARY KEY,
    LocationCode TEXT NOT NULL
);

-- Course Meetings table
DROP TABLE IF EXISTS COURSE_MEETINGS;
CREATE TABLE COURSE_MEETINGS (
    CourseMeetingID INTEGER PRIMARY KEY,
    CourseOfferingID INTEGER NOT NULL,
    LocationID INTEGER NOT NULL,
    StartDateTime TEXT NOT NULL,
    EndDateTime TEXT NOT NULL,
    FOREIGN KEY (CourseOfferingID) REFERENCES COURSE_OFFERINGS(CourseOfferingID),
    FOREIGN KEY (LocationID) REFERENCES LOCATIONS(LocationID)
);

-- A conversion table for matching the term to the corresponding catalog year
-- Catalogs are not available for the first several years
DROP TABLE IF EXISTS TERM_CATALOG_YEAR;
CREATE TABLE TERM_CATALOG_YEAR (
    CatalogYear TEXT NOT NULL,
    Term TEXT NOT NULL
);

INSERT INTO TERM_CATALOG_YEAR (Term, CatalogYear) VALUES 
('Fall2017','2017_2018'),('Winter2018','2017_2018'),('Spring2018','2017_2018'),('Summer2018','2017_2018'),('Fall2018','2018_2019'),
('Winter2019','2018_2019'),('Spring2019','2018_2019');

 * sqlite:///CourseData.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
7 rows affected.


[]

# Extracting Data 

In [4]:
#taken from example 
# Catalog Data
catalog_years = ['2017_2018', '2018_2019']

for cat_year in catalog_years:
    filepath = 'SourceData/Catalogs/CourseCatalog'+cat_year+'.csv'
    data = pd.read_csv(filepath)
    data['cat_year'] = cat_year
    data.to_sql('IMPORT_CATALOG_COURSES',conn,if_exists='append',index=False)

In [5]:
#continued taken from example
# Course Offering and Course Meeting Data
terms = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018',
         'Spring2015','Spring2016','Spring2017','Spring2018','Spring2019',
         'SpringBreak2017',
         'Summer2015','Summer2016','Summer2017','Summer2018',
         'Winter2015','Winter2016','Winter2017','Winter2018']

for term in terms:
    filepath = 'SourceData/'+term+'/courses.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_COURSE_OFFERINGS',conn,if_exists='append',index=False) 
    
    filepath = 'SourceData/'+term+'/course_meetings.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_COURSE_MEETINGS',conn,if_exists='append',index=False)

# SQL Commands 

In [6]:
%%sql
-- Record Counts for Catalog Courses
SELECT 
    (SELECT Count(*) FROM IMPORT_CATALOG_COURSES) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_CATALOG_COURSES)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
4440,4440


In [7]:
%%sql
-- Record Counts for Catalog Courses
SELECT 
    (SELECT Count(*) FROM IMPORT_CATALOG_COURSES) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_CATALOG_COURSES)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
4440,4440


In [8]:
%%sql 
-- Record Counts for Course Meetings
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_MEETINGS) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_MEETINGS)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
284907,284847


In [9]:
%%sql 
SELECT term,crn,location, day, start 
FROM IMPORT_COURSE_MEETINGS
GROUP BY term,crn,location, day, start
HAVING COUNT(*)>1
LIMIT 5;

 * sqlite:///CourseData.db
Done.


term,crn,location,day,start
Fall2014,73073,MCA 102,M,2014-09-08T18:30:00
Fall2014,73073,MCA 102,M,2014-09-15T18:30:00
Fall2014,73073,MCA 102,M,2014-09-22T18:30:00
Fall2014,73073,MCA 102,M,2014-09-29T18:30:00
Fall2014,73073,MCA 102,M,2014-10-06T18:30:00


# Load Data to Tables

In [10]:
%%sql
DELETE FROM INSTRUCTORS;

INSERT INTO INSTRUCTORS (Name)
SELECT DISTINCT primary_instructor
FROM import_course_offerings 
WHERE primary_instructor <> 'TBA' AND primary_instructor NOT LIKE '%/%';

 * sqlite:///CourseData.db
0 rows affected.
1095 rows affected.


[]

In [11]:
%%sql
DELETE FROM PROGRAMS;

INSERT INTO PROGRAMS (ProgramCode,ProgramName)
SELECT DISTINCT program_code,program_name 
FROM IMPORT_CATALOG_COURSES
ORDER BY program_code;


 * sqlite:///CourseData.db
0 rows affected.
83 rows affected.


[]

In [12]:
%%sql 
DELETE FROM CATALOG_COURSES;

INSERT INTO CATALOG_COURSES (CatalogYear,ProgramID,CatalogID,CourseTitle,Credits,Prereqs,Coreqs,Fees,Attributes,Description)
SELECT DISTINCT cat_year, ProgramID,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
FROM IMPORT_CATALOG_COURSES 
    JOIN PROGRAMS ON (program_code = ProgramCode);

 * sqlite:///CourseData.db
0 rows affected.
4440 rows affected.


[]

In [13]:
%%sql
DELETE FROM COURSE_OFFERINGS;

INSERT INTO COURSE_OFFERINGS (CourseID,Term,CRN,CatalogID,Section,Credits,Title,Timecodes,PrimaryInstructorID,Capacity, Actual, Remaining)
SELECT DISTINCT CourseID,term,crn,catalog_id,section,import_course_offerings.credits,title,timecodes,InstructorID,cap,act,rem 
FROM import_course_offerings 
    LEFT JOIN INSTRUCTORS ON (primary_instructor=INSTRUCTORS.Name)
    LEFT JOIN TERM_CATALOG_YEAR USING (Term)
    LEFT JOIN CATALOG_COURSES ON (catalog_id = CatalogID AND CATALOG_COURSES.CatalogYear = TERM_CATALOG_YEAR.CatalogYear)
;

 * sqlite:///CourseData.db
0 rows affected.
15937 rows affected.


[]

In [14]:
%%sql 
INSERT INTO LOCATIONS (LocationCode)
SELECT DISTINCT Location 
FROM import_course_meetings
ORDER BY Location

 * sqlite:///CourseData.db
207 rows affected.


[]

In [15]:
%%sql
DELETE FROM COURSE_MEETINGS;

INSERT INTO COURSE_MEETINGS (CourseOfferingID,LocationID,StartDateTime,EndDateTime)
SELECT DISTINCT COURSE_OFFERINGS.CourseOfferingID, LocationID,`Start`,`End`
FROM import_course_meetings 
    JOIN COURSE_OFFERINGS USING (Term,CRN)
    LEFT JOIN LOCATIONS ON (import_course_meetings.Location = LOCATIONS.LocationCode);

 * sqlite:///CourseData.db
0 rows affected.
284847 rows affected.


[]

# 5- integrity checks - new notebook

# 6 - empty 

In [16]:
%%sql
-- Delete raw data
DELETE FROM IMPORT_CATALOG_COURSES;
DELETE FROM IMPORT_COURSE_OFFERINGS;
DELETE FROM IMPORT_COURSE_MEETINGS;

 * sqlite:///CourseData.db
Done.
15937 rows affected.
284907 rows affected.


[]

In [17]:
%%sql
-- Drop the tables
DROP TABLE IMPORT_CATALOG_COURSES;
DROP TABLE IMPORT_COURSE_OFFERINGS;
DROP TABLE IMPORT_COURSE_MEETINGS;

 * sqlite:///CourseData.db
Done.
Done.
Done.


[]

In [18]:
%%sql
-- Force SQLite to rebuild the database file to minimize file size
vacuum;

 * sqlite:///CourseData.db
Done.


[]

In [19]:
%%sql
SELECT *
FROM PROGRAMS
LIMIT 5;

 * sqlite:///CourseData.db
Done.


ProgramID,ProgramCode,ProgramName
1,AC,Accounting
2,AE,Applied Ethics
3,AH,Art History
4,AN,Asian Studies
5,AR,Arabic


In [20]:
%%sql
SELECT *
FROM INSTRUCTORS
LIMIT 5;

 * sqlite:///CourseData.db
Done.


InstructorID,Name
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos


In [21]:
%%sql 
SELECT *
FROM COURSE_OFFERINGS
LIMIT 5;

 * sqlite:///CourseData.db
Done.


CourseOfferingID,CourseID,CatalogID,Term,CRN,Section,Credits,Title,Timecodes,PrimaryInstructorID,Capacity,Actual,Remaining
1,None,AC 0011,Fall2014,70384,C01,3.0,Introduction to Financial Accounting,['TF 0800am-0915am 09/02-12/08 DSB 105'],1,0,31,-31
2,None,AC 0011,Fall2014,70385,C02,3.0,Introduction to Financial Accounting,['TF 0930am-1045am 09/02-12/08 DSB 105'],1,0,31,-31
3,None,AC 0011,Fall2014,70382,C03,3.0,Introduction to Financial Accounting,['TF 1230pm-0145pm 09/02-12/08 DSB 105'],1,0,31,-31
4,None,AC 0011,Fall2014,70291,C04,3.0,Introduction to Financial Accounting,['MR 1100am-1215pm 09/02-12/08 DSB 111'],2,0,29,-29
5,None,AC 0011,Fall2014,70350,C05,3.0,Introduction to Financial Accounting,['MR 1230pm-0145pm 09/02-12/08 DSB 111'],2,0,30,-30
